In [ ]:
using vOptGeneric, JuMP, Gurobi, DataFrames, VegaLite, Serialization, PlotlyJS, JMcDM, JSON

include("../src/utils/plot.jl")

In [ ]:
# Daten Laden
result = deserialize("../results/result.jls")[:,1:9]
benchmark = deserialize("../results/benchmark.jls")

# Neue Spalte mit laufenden Nummern hinzufügen
result.ID = 1:nrow(result)
benchmark.ID = 1:nrow(benchmark)

# Result und Benchmark auf beste Pareto Front Filtern
filtered_result = filter(row -> row.Rang == 1.0, result)
filtered_benchmark = filter(row -> row.Rang == 1.0, benchmark)
filter!(row -> row.ID != 37, filtered_benchmark) # der Wert scheint ein falsch berechneter Ausreiser zu sein deshalb wird der hier entfernt
filtered_result = combine(groupby(filtered_result, [:Dauer, :Kosten, :Belastung, :Modus]), first)
filtered_benchmark = combine(groupby(filtered_benchmark, [:Dauer, :Kosten, :Belastung, :Modus]), first)

In [ ]:
# Merging result and Benchmark into one DataFrame

result_merged = select(filtered_result, :Dauer, :Kosten, :Belastung, :Modus, :ID) 
result_merged = unique(combine(groupby(result_merged, [:Dauer, :Kosten, :Belastung, :Modus]), first))
result_merged[:, :Source] .= "Result"

benchmark_merged = select(filtered_benchmark, :Dauer, :Kosten, :Belastung, :Modus, :ID) 
benchmark_merged[:, :Source] .= "Benchmark"

# DataFrames zusammenführen
merged_df = vcat(result_merged, benchmark_merged)

In [ ]:
# Rang hinzufügen
merged_df[:, :Rang] = non_dominated_sort(merged_df)
df = filter(row -> row.Rang == 1, merged_df)
result_df = filter(row -> row.Source == "Result", df)
benchmark_df = filter(row -> row.Source == "Benchmark", df)

println("Benchmark: ", nrow(benchmark_df), " nicht dominierte Punkte; ", nrow(benchmark_merged) - nrow(benchmark_df), " dominierte Punkte")
println("Result: ",  nrow(result_df), " nicht dominierte Punkte; ", nrow(result_merged) - nrow(result_df), " dominierte Punkte")
println("Verhältnis: ", round(nrow(result_df) / nrow(df)*100, digits = 1), "% der Lösungen aus NSGA-II")

In [ ]:
plt = plot_3d_scatter(filtered_benchmark)
display(plt)

In [ ]:
plt = plot_3d_scatter(filtered_result, nothing, "red")
display(plt)

In [ ]:
#Anzeigen aller Punkte von Benchmark und Result
plt = plot_3d_scatter(filtered_benchmark, filtered_result)
display(plt)

In [ ]:
plt = plot_3d_scatter_zoom(filtered_benchmark, filtered_result)
display(plt)

In [ ]:
plt = plot_3d_scatter(benchmark_df, result_df)
display(plt)

In [ ]:
df = merged_df

# Zuerst gruppierst du den DataFrame nach Dauer, Kosten und Modus
grouped_df = groupby(df, [:Dauer, :Kosten, :Modus])

# Nun pivotierst du die Daten basierend auf der Spalte 'Source', 
# wobei die Spalte 'Belastung' als Wert dient, der in den neuen Spalten angezeigt wird
pivoted_df = unstack(df, [:Dauer, :Kosten, :Modus], :Source, :Belastung)

function calc_percentage_diff(result, benchmark)
    if ismissing(result) || ismissing(benchmark) || benchmark == 0.0
        return missing
    else
        return ((result - benchmark) / benchmark) * 100
    end
end

# Neue Spalte mit der prozentualen Abweichung hinzufügen
pivoted_df[!, :Prozentuale_Abweichung] = calc_percentage_diff.(pivoted_df.Result, pivoted_df.Benchmark)

# Den DataFrame nach der Dauer sortieren
sort!(pivoted_df, :Dauer)

In [ ]:
using Statistics  # Für die mean Funktion

# Berechne den Durchschnitt der prozentualen Abweichung, ignoriere missing-Werte
durchschnitt = mean(skipmissing(pivoted_df.Prozentuale_Abweichung))
medianwert = median(skipmissing(pivoted_df.Prozentuale_Abweichung))
max = maximum(skipmissing(pivoted_df.Prozentuale_Abweichung))
min = minimum(skipmissing(pivoted_df.Prozentuale_Abweichung))
missing_result = count(ismissing, pivoted_df.Result)
missing_benchmark = count(ismissing, pivoted_df.Benchmark)
quantil_90 = round(quantile(skipmissing(pivoted_df.Prozentuale_Abweichung), 0.90), digits=4)


# Ergebnis anzeigen
println("Der Durchschnitt der prozentualen Abweichung beträgt: ", durchschnitt, "%")
println("Der Median der prozentualen Abweichung beträgt: ", medianwert, "%")
println("Maximale Abweichung: ", max, "%")
println("Minimale Abweichung: ", min, "%")
println("Anzahl der Lösungen, die in Benchmark vorhanden sind aber nicht in Result: ", missing_result)
println("Anzahl der Lösungen, die in Result vorhanden sind aber nicht in Benchmark: ", missing_benchmark)
println("Anzahl der Lösungspaare: ", nrow(pivoted_df) - missing_result - missing_benchmark)
println("Anzahl der gesamten Lösungen: ", nrow(pivoted_df))
println("90% Quantil: ", quantil_90, "%")

In [ ]:
filtered_df = filter(row -> row.Prozentuale_Abweichung !== missing, pivoted_df)
reordered_df = select(filtered_df, :Dauer, :Modus, :Kosten, :Benchmark, :Result, :Prozentuale_Abweichung)
reordered_df.Prozentuale_Abweichung = string.(round.(filtered_df.Prozentuale_Abweichung, digits=2)) .* "%"
reordered_df